In [68]:
# https://rosettacode.org/wiki/Reverse_the_gender_of_a_string#Python
 
import re
male2female=u"""maleS femaleS, maleness femaleness,
him her, himself herself, his her, his hers, he she,
Mr Mrs, Mister Missus, Ms Mr, Master Miss, Master Mistress,
uncleS auntS, nephewS nieceS, sonS daughterS, grandsonS granddaughterS,
brotherS sisterS, man woman, men women, boyS girlS, paternal maternal,
grandfatherS grandmotherS,
husband wife, husbands wives, fatherS motherS, bridegroomS brideS, widowerS widowS,
KingS QueenS,PrinceS PrincessES,
Lord Lady, Lords Ladies,ladS lassES, sir madam, gentleman lady, gentlemen ladies,
godS goddessES, heroS heroineS, landlord landlady, landlords landladies, 
manservantS maidservantS, actorS actressES,
boyfriendS girlfriendS, dogS bitchES, daddy mommy, dadS momS"""
 
re_newline=re.compile(r",[ \n]*")
male2female_splitted=[ token.split(" ") for token in re_newline.split(male2female) ]
 
re_plural=re.compile("E*S$")
re_ES=re.compile("ES$")
 
def gen_pluralize(m,f):
# do plurals first 
  yield re_plural.sub("",m),re_plural.sub("",f)
  yield re_ES.sub("es",m),re_ES.sub("es",f)
  yield re_plural.sub("s",m),re_plural.sub("s",f)
 
def gen_capitalize_pluralize(m,f):
  for m,f in gen_pluralize(m,f):
    yield m.capitalize(), f.capitalize()
    yield m,f

# converts male pronouns to female and female to male
def gen_switch(male_to_female=True, female_to_male=True):
  switch={}
  words=[]

  for male,female in male2female_splitted:
    # for xy, xx in gen_capitalize_pluralize(male,female):
    for xy, xx in gen_pluralize(male,female):
      if male_to_female and xy not in switch: 
        switch[xy]=xx
        words.append(xy)
      if female_to_male and xx not in switch: 
        switch[xx]=xy
        words.append(xx)

  words="|".join(words)
  re_word = re.compile(r"\b("+words+r")\b")
  return re_word, switch

def rev_gender(text, re_word, switch):
  text=re_word.split(text)
  return "".join([ word+switch[gen] for word,gen in zip(text[::2],text[1::2])]) + text[-1]

In [47]:
text=u'''When a new-hatched savage running wild about her native
woodlands in a grass clout, followed by the nibbling goats, as if
he were a green sapling; There was excellent
blood in his veins-royal stuff; though sadly vitiated, I fear,
by the cannibal propensity he nourished in his untutored youth.'''
 
print(rev_gender(text))

When a new-hatched savage running wild about him native
woodlands in a grass clout, followed by the nibbling goats, as if
he were a green sapling; There was excellent
blood in his veins-royal stuff; though sadly vitiated, I fear,
by the cannibal propensity he nourished in his untutored youth.


In [71]:
text = 'shout out to my # babydaddy # fathersday'
re_word, switch = gen_switch()
print(rev_gender(text.lower(), re_word, switch))

shout out to my # babydaddy # fathersday


In [78]:
from datasets import load_dataset
tweets_dataset = load_dataset('tweets_hate_speech_detection')

Using custom data configuration default
Reusing dataset tweets_hate_speech_detection (C:\Users\khair\.cache\huggingface\datasets\tweets_hate_speech_detection\default\0.0.0\c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2)
100%|██████████| 1/1 [00:00<00:00, 332.17it/s]


In [79]:
tweets_dataset['train']['label'].count(1), tweets_dataset['train']['label'].count(0)

(2242, 29720)

In [65]:

tweets_datasets = tweets_dataset['train'].train_test_split(test_size=0.2)
tweets_datasets = tweets_datasets.rename_column("tweet", "text")

C:\Users\khair\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using custom data configuration default


Dataset tweets_hate_speech_detection downloaded and prepared to C:\Users\khair\.cache\huggingface\datasets\tweets_hate_speech_detection\default\0.0.0\c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 83.35it/s]


In [72]:
dataset = tweets_datasets

dataset['test_gender_swapped'] = dataset['test']
dataset['test_male_to_female'] = dataset['test']
dataset['test_female_to_male'] = dataset['test']

In [73]:
from tqdm.auto import tqdm

re_word_both, switch_both = gen_switch(male_to_female=True, female_to_male=True)
re_word_f2m, switch_f2m = gen_switch(male_to_female=False, female_to_male=True)
re_word_m2f, switch_m2f = gen_switch(male_to_female=True, female_to_male=False)

for index in tqdm(range(len(dataset['test']['text']))):
    text = dataset['test']['text'][index]

    dataset['test_gender_swapped']['text'][index] = rev_gender(text, re_word_both, switch_both)
    dataset['test_female_to_male']['text'][index] = rev_gender(text, re_word_f2m, switch_f2m)
    dataset['test_male_to_female']['text'][index] = rev_gender(text, re_word_m2f, switch_m2f)

100%|██████████| 6393/6393 [16:08<00:00,  6.60it/s]


In [74]:
dataset.save_to_disk('tweet-dataset')

Flattening the indices: 100%|██████████| 7/7 [00:00<00:00, 107.69ba/s]
Loading cached processed dataset at C:\Users\khair\.cache\huggingface\datasets\tweets_hate_speech_detection\default\0.0.0\c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2\cache-c8c0e60c236932e7.arrow
Loading cached processed dataset at C:\Users\khair\.cache\huggingface\datasets\tweets_hate_speech_detection\default\0.0.0\c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2\cache-c8c0e60c236932e7.arrow
Loading cached processed dataset at C:\Users\khair\.cache\huggingface\datasets\tweets_hate_speech_detection\default\0.0.0\c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2\cache-c8c0e60c236932e7.arrow


(1806, 23763)

In [75]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 25569
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 6393
    })
    test_gender_swapped: Dataset({
        features: ['label', 'text'],
        num_rows: 6393
    })
    test_male_to_female: Dataset({
        features: ['label', 'text'],
        num_rows: 6393
    })
    test_female_to_male: Dataset({
        features: ['label', 'text'],
        num_rows: 6393
    })
})